In [7]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 값과 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")  # 해산도를 높어서 축소 효과를 낼다.
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%8A%A4%EB%85%B8%EC%9A%B0%ED%8F%AD%EC%8A%A4%20%EB%B1%85%EB%B1%85%EC%A0%90/place/36503902?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 검색
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 검색
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("\uac00\uac8c \uc774\ub984:", store_name)
    print("\uc5c5종 \uce74\ud14c\uace0\ub9ac:", category)
    print("\ubcc4\uc810:", rating)
    print("\ubc29\ubb38\uc790 \ub9ac\ubdf0:", visitor_review)
    print("\ube14\ub85c\uadf8 \ub9ac\ubdf0:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()  # 중복 확인을 위한 set
    no_more_button_count = 0  # 더보기 버튼이 없을 때의 카운트

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:  # 최대 3번 재시도
                try:
                    # 리뷰 리스트를 매번 새로 가져옴
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    # 스크롤을 조정하여 요소가 화면에 나타나도록 함
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)  # 스크롤 후 약간의 지연을 줌

                    # 리뷰 텍스트, 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    # 사진이 있을 때
                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        # 사진이 없을 때
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    # 중복 확인 및 저장
                    if review_text and (review_text, review_date) not in review_text_set:
                        reviews.append((review_date, review_text))  # 사용자 이름 제외
                        review_text_set.add((review_text, review_date))  # 중복 확인을 위해 텍스트와 날짜 추가
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break  # 성공하면 루프 탈출
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)  # 재시도 전 약간의 지연

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
            no_more_button_count = 0  # 버튼 클릭 시 카운트 초기화
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollBy(0, 300);")  # 조금씩 스크롤
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
            no_more_button_count += 1

        # 더보기 버튼이 5번 연속으로 없으면 저장하고 종료
        if no_more_button_count >= 5:
            print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    # 예외가 발생할 경우 메시지 출력
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    # 'reviews' 리스트의 각 항목이 ('Date', 'Review') 2개의 요소로 되어있다는 것을 확실히 합니다.
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 스노우폭스 뱅뱅점
업종 카테고리: 초밥,롤
별점: 4.18
방문자 리뷰: 968
블로그 리뷰: 263
리뷰 탭 클릭 완료

리뷰 총 개수 가져오기
총 리뷰 수: 304
리뷰 1: 연어찐 찐맛집.. 한 번 가보고 중독되어단골된 1인입니다
너무 신선하고 매장관리잘되어잇고 사장님도 직원들도 친절.
사진보니까 또먹고싶네요.

스노우폭스는 꽃이아니라 도시락맛집임 (2024년 11월 5일 화요일)

리뷰 2: 서울 올 때마다 방문하게 되는 곳이네요 ◡̎
오늘은 밥이 설익은 느낌이었어요.. 아쉬운 마음도 있었지만 연어는 넘넘 맛있었고, 샐러드도 든든하게 잘 먹었습니다 🙌🏻 (2024년 10월 13일 일요일)

리뷰 3: ✔️ 뱅뱅사거리 도시락 맛집, 혼밥 맛집 찾으시는 분들 아묻따 스노우폭스 뱅뱅점 추천합니다
캘리포니아롤부터 샐러드 포케 오니기리 덮밥 스프 샌드위치 등 진짜 엄청 다양한 메뉴가 있는다
심지어 하나같이 다 맛나요.
특히 연어 요리들이 많은데 연어 들어간건 무조건 맛있어요 요기 ㅎㅎ
오랜만에 먹었더니 더 맛나고 감동적이네요~ >< 사장님 많이 파세요 (2024년 5월 22일 수요일)

리뷰 4: 혼밥하기 너무좋아용 밖에 볼일있을때나 약속전에 배고플때 간단히 밥먹기 넘 좋은곳! 뱅뱅사거리 교차로 바로 앞이에용 진짜 자주옴니당👍🏻스시좋아하는분들 고고 (2024년 7월 16일 화요일)

리뷰 5: 재료가 신선해요! 과일도 맛나요 (2024년 7월 19일 금요일)

리뷰 6: 연어초밥 맛있어요! (2024년 9월 15일 일요일)

리뷰 7: 와 너무 맛있습니다! 양도 많고 가격도 저렴하면서~ 이렇게 맛있다니요!!! 전 롤도 완전 제스탈!!!! 놀러갈때 도시락으로 사갔더니 인기 만점에 센스쟁이 됐구요~ 그때부터 빠져서 아침출근할때 하나씩 사들고 갑니다👍🏻 커피도 저렴해 같이 먹기 딱!! 개인적으로 롤….진짜 너무 맛있습니다 (2024년 1월 23일 화요일)

리뷰 8: 한국브랜드의 세계적인 규모의 글로벌 도시락체인점 스노우폭스 뱅뱅점

연어를 주 재료로 사

KeyboardInterrupt: 

In [10]:
import os
print("Current working directory:", os.getcwd())


Current working directory: C:\fintech_service\final_project


# 찐 최종

In [12]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 값과 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()  # 중복 확인을 위한 set
    no_more_button_count = 0  # 더보기 버튼이 없을 때의 카운트

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:  # 최대 3번 재시도
                try:
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)

                    # 리뷰 텍스트, 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    if review_text and (review_text, review_date) not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add((review_text, review_date))
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)

        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
            no_more_button_count = 0
        except (NoSuchElementException, TimeoutException):
            driver.execute_script("window.scrollBy(0, 300);")
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
            no_more_button_count += 1

        if no_more_button_count >= 5:
            print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


가게 이름: 큐뮬러스
업종 카테고리: 샌드위치
별점: 방문자 리뷰 406
방문자 리뷰: 363
블로그 리뷰: 없음
리뷰 탭 클릭 완료

리뷰 총 개수 가져오기
총 리뷰 수: 223
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일)

리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일)

리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일)

리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일)

리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일)

리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일)

리뷰 7: 성수동 때 부터 애정하는 큐뮬러스. 큐뮬러스는

KeyboardInterrupt: 

# 한 번 해보기

In [15]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 값과 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 리뷰 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()  # 중복 확인을 위한 set
    no_more_button_count = 0  # 더보기 버튼이 없을 때의 카운트

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:  # 최대 3번 재시도
                try:
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)

                    # 리뷰 텍스트, 날짜, 사람 이름 경로
                    review_text_xpath_options = [
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/a'
                    ]

                    review_date_xpath_options = [
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'
                    ]

                    reviewer_name_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[1]/a[2]/div[1]/span/span'

                    review_text_element = None
                    review_date_element = None

                    # 리뷰 텍스트 요소 찾기
                    for xpath in review_text_xpath_options:
                        try:
                            review_text_element = driver.find_element(By.XPATH, xpath)
                            if review_text_element:
                                break
                        except NoSuchElementException:
                            continue

                    # 리뷰 날짜 요소 찾기
                    for xpath in review_date_xpath_options:
                        try:
                            review_date_element = driver.find_element(By.XPATH, xpath)
                            if review_date_element:
                                break
                        except NoSuchElementException:
                            continue

                    reviewer_name_element = driver.find_element(By.XPATH, reviewer_name_xpath)

                    if review_text_element and review_date_element and reviewer_name_element:
                        review_text = review_text_element.text.strip()
                        review_date = review_date_element.text.strip()
                        reviewer_name = reviewer_name_element.text.strip()

                        # 중복 확인에 이름 포함
                        if review_text and (review_text, review_date, reviewer_name) not in review_text_set:
                            reviews.append((review_date, review_text))  # 저장 시 이름은 제외
                            review_text_set.add((review_text, review_date, reviewer_name))
                            review_count += 1
                            print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                        else:
                            print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)

        # '더보기' 버튼 처리
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
            no_more_button_count = 0
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없는 경우 스크롤 다운
            driver.execute_script("window.scrollBy(0, 300);")
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
            no_more_button_count += 1

        if no_more_button_count >= 5:
            print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 223
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일)

리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일)

리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일)

리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일)

리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일)

리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일)

리뷰 7: 성수동 때 부터 애정하는 큐뮬러스. 큐뮬러스는 오늘도 감동적인 맛💓☘️🩵 (2024년 4월 9일 화요일)

리뷰 8: +4
개의 리뷰가 더 있습니다
펼쳐보기 (2024년 6월 26일 수

KeyboardInterrupt: 

위에 되긴함 저장 문제

In [18]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()
    no_more_button_count = 0

    while review_count < total_reviews and review_count < 1000:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:
                try:
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)

                    # 리뷰 텍스트, 날짜, 사람 이름 경로
                    review_text_xpath_options = [
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/a'
                    ]

                    review_date_xpath_options = [
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'
                    ]

                    reviewer_name_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[1]/a[2]/div[1]/span/span'

                    review_text_element = None
                    review_date_element = None

                    for xpath in review_text_xpath_options:
                        try:
                            review_text_element = driver.find_element(By.XPATH, xpath)
                            if review_text_element:
                                break
                        except NoSuchElementException:
                            continue

                    for xpath in review_date_xpath_options:
                        try:
                            review_date_element = driver.find_element(By.XPATH, xpath)
                            if review_date_element:
                                break
                        except NoSuchElementException:
                            continue

                    reviewer_name_element = driver.find_element(By.XPATH, reviewer_name_xpath)

                    if review_text_element and review_date_element and reviewer_name_element:
                        review_text = review_text_element.text.strip()
                        review_date = review_date_element.text.strip()
                        reviewer_name = reviewer_name_element.text.strip()

                        if review_text and (review_text, review_date, reviewer_name) not in review_text_set:
                            reviews.append((review_date, review_text))
                            review_text_set.add((review_text, review_date, reviewer_name))
                            review_count += 1
                            print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                        else:
                            print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)

        # '더보기' 버튼 처리
        if review_count < total_reviews:
            try:
                bottom_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
                driver.execute_script("arguments[0].click();", bottom_more_button)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
                )
                print("하단 '더보기' 버튼 클릭 완료")
                no_more_button_count = 0
            except (NoSuchElementException, TimeoutException):
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
                no_more_button_count += 1

            if no_more_button_count >= 5:
                print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
                break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


가게 이름: 큐뮬러스
업종 카테고리: 샌드위치
별점: 방문자 리뷰 406
방문자 리뷰: 363
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 223
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일)

리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일)

리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일)

리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일)

리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일)

리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일)

리뷰 7: 성수동 때 부터 애정하는 큐뮬러스. 큐뮬러스는 오늘도 감동적인 맛💓☘️

# 위에가 일단 맞는거...? 하

In [19]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()
    no_more_button_count = 0

    while review_count < total_reviews and review_count < 1000:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:
                try:
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)

                    # 리뷰 텍스트, 날짜, 사람 이름 경로
                    review_text_xpath_options = [
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a',  # 추가된 XPath 옵션
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[4]/a',  # 알려주신 위치
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/a'
                    ]
                    
                    review_date_xpath_options = [
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]',
                        f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'
                    ]
                    
                    reviewer_name_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[1]/a[2]/div[1]/span/span'
                    review_text_element = None
                    review_date_element = None

                    for xpath in review_text_xpath_options:
                        try:
                            review_text_element = driver.find_element(By.XPATH, xpath)
                            if review_text_element:
                                break
                        except NoSuchElementException:
                            continue

                    for xpath in review_date_xpath_options:
                        try:
                            review_date_element = driver.find_element(By.XPATH, xpath)
                            if review_date_element:
                                break
                        except NoSuchElementException:
                            continue

                    reviewer_name_element = driver.find_element(By.XPATH, reviewer_name_xpath)

                    if review_text_element and review_date_element and reviewer_name_element:
                        review_text = review_text_element.text.strip()
                        review_date = review_date_element.text.strip()
                        reviewer_name = reviewer_name_element.text.strip()

                        if review_text and (review_text, review_date, reviewer_name) not in review_text_set:
                            reviews.append((review_date, review_text))
                            review_text_set.add((review_text, review_date, reviewer_name))
                            review_count += 1
                            print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                        else:
                            print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)

        # '더보기' 버튼 처리
        if review_count < total_reviews:
            try:
                bottom_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
                driver.execute_script("arguments[0].click();", bottom_more_button)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
                )
                print("하단 '더보기' 버튼 클릭 완료")
                no_more_button_count = 0
            except (NoSuchElementException, TimeoutException):
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
                no_more_button_count += 1

            if no_more_button_count >= 5:
                print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
                break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


가게 이름: 큐뮬러스
업종 카테고리: 샌드위치
별점: 방문자 리뷰 406
방문자 리뷰: 363
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 223
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일)

리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일)

리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일)

리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일)

리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일)

리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일)

리뷰 7: 성수동 때 부터 애정하는 큐뮬러스. 큐뮬러스는 오늘도 감동적인 맛💓☘️

In [24]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()
    no_more_button_count = 0

    while review_count < total_reviews and review_count < 1000:
        # 기본 리뷰 경로에서 리뷰 요소를 가져옴
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        
        # '개의 리뷰가 더 있습니다' 텍스트가 있으면 추가 리뷰를 수집
        try:
            additional_reviews_text = driver.find_elements(By.XPATH, "//*[contains(text(), '개의 리뷰가 더 있습니다')]")
            if additional_reviews_text:
                print("추가 리뷰를 찾음, 다른 경로로 리뷰 가져오기 시작...")
                review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li')

            for i, review_element in enumerate(review_elements, start=1):
                try:
                    # 사진이 있는 리뷰와 없는 리뷰에 따른 XPath 설정
                    review_text_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'

                    # 사진이 없는 리뷰는 다른 XPath 사용
                    if not driver.find_elements(By.XPATH, review_text_xpath):
                        review_text_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                        review_date_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    review_text = driver.find_element(By.XPATH, review_text_xpath).text.strip()
                    review_date = driver.find_element(By.XPATH, review_date_xpath).text.strip()

                    # 중복 방지 및 저장
                    if (review_text, review_date) not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add((review_text, review_date))
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})")
                except NoSuchElementException:
                    continue

            # '더보기' 버튼 처리
            try:
                bottom_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
                driver.execute_script("arguments[0].click();", bottom_more_button)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
                )
                print("하단 '더보기' 버튼 클릭 완료")
                no_more_button_count = 0
            except (NoSuchElementException, TimeoutException):
                print("하단 '더보기' 버튼이 없어졌습니다.")
                break

            if no_more_button_count >= 5:
                print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰 수집을 종료합니다.")
                break

        except Exception as e:
            print(f"오류 발생: {e}")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


가게 이름: 큐뮬러스
업종 카테고리: 샌드위치
별점: 방문자 리뷰 406
방문자 리뷰: 363
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 223
추가 리뷰를 찾음, 다른 경로로 리뷰 가져오기 시작...
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일)
리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일)
리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일)
리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일)
리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일)
리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일)
리뷰 7: 성수동 때 부터 애정하는

# 리뷰 하나하나 확인하는중...

In [26]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()
    no_more_button_count = 0

    # 처음부터 총리뷰 개수까지 다 수집하도록 설정
    while review_count < total_reviews and review_count < 1000:
        # 리뷰 인덱스가 총 리뷰 개수만큼 반복하여 모든 리뷰 수집
        for i in range(1, total_reviews + 1):
            try:
                # 사진이 있는 리뷰와 없는 리뷰에 따른 XPath 설정
                review_text_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'

                
                review_date_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'

                # 사진이 없는 리뷰는 다른 XPath 사용
                if not driver.find_elements(By.XPATH, review_text_xpath):
                    review_text_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                review_text = driver.find_element(By.XPATH, review_text_xpath).text.strip()
                review_date = driver.find_element(By.XPATH, review_date_xpath).text.strip()

                # 중복 방지 및 저장
                if (review_text, review_date) not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add((review_text, review_date))
                    review_count += 1
                    print(f"리뷰 {review_count}: {review_text} ({review_date})")
            except NoSuchElementException:
                continue

        # '더보기' 버튼 처리
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(2)  # 충분히 대기
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
            no_more_button_count = 0
        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

        if no_more_button_count >= 5:
            print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰 수집을 종료합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


가게 이름: 큐뮬러스
업종 카테고리: 샌드위치
별점: 방문자 리뷰 406
방문자 리뷰: 363
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 223
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일)
리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일)
리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일)
리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일)
리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일)
리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일)
리뷰 7: 성수동 때 부터 애정하는 큐뮬러스. 큐뮬러스는 오늘도 감동적인 맛💓☘️🩵 (202

In [ ]:
//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[8]/div[4]/a
//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[22]/div[4]/a
//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[25]/div[4]/a
//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[31]/div[4]/a

In [4]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        time.sleep(2)
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 스크롤을 조금씩 내려서 리뷰 섹션이 로드되도록 함
    scroll_attempts = 0
    while scroll_attempts < 10:
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)
        try:
            review_count_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
            )
            total_reviews = int(review_count_element.text.replace(',', ''))
            print("총 리뷰 수:", total_reviews)
            break
        except TimeoutException:
            scroll_attempts += 1
            print("스크롤 시도:", scroll_attempts)
    
    if scroll_attempts == 10:
        print("리뷰 섹션을 찾을 수 없습니다. 스크롤 시도 종료")
        driver.quit()
        exit()

    # 리뷰 수집
    collected_reviews = []
    current_count = 0

    while current_count < total_reviews:
        # 현재 페이지의 리뷰 요소를 수집
        review_elements = driver.find_elements(By.CSS_SELECTOR, "div.place_section.k1QQ5 div.place_section_content ul li")

        for review_element in review_elements:
            try:
                # 리뷰 내용 추출
                review_text_element = WebDriverWait(review_element, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.pui__vn15t2 > a.pui__xtsQN-"))
                )
                review_text = review_text_element.text

                # 방문일자 추출
                review_date_element = WebDriverWait(review_element, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span.pui_blind"))
                )
                review_date = review_date_element.text

                # 리뷰 데이터 추가
                collected_reviews.append({"날짜": review_date, "리뷰 내용": review_text})
                current_count += 1
            except NoSuchElementException as e:
                print("리뷰 또는 날짜 추출 오류:", e)

        print(f"현재 수집한 리뷰 수: {current_count}")

        # 수집한 리뷰 수가 총 리뷰 수와 같으면 반복 종료
        if current_count >= total_reviews:
            break

        # '더 보기' 버튼 클릭하여 추가 리뷰 로드
        try:
            more_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.NSTUp > div > a > span.Teltc")))
            more_button.click()
            time.sleep(2)  # 다음 ul 로딩 대기
            driver.execute_script("window.scrollBy(0, 400);")  # 더보기 후 스크롤
        except (NoSuchElementException, TimeoutException) as e:
            print("더 보기 버튼을 찾을 수 없거나 클릭할 수 없습니다.", e)
            break

    # 리뷰 데이터를 DataFrame으로 변환 후 CSV 파일로 저장
    reviews_df = pd.DataFrame(collected_reviews)
    reviews_df.to_csv("reviews.csv", index=False, encoding="utf-8-sig")

    print("리뷰 수집 완료 및 CSV 저장 완료!")
    print(f"총 수집한 리뷰 수: {len(collected_reviews)}")
    # driver.quit()

except Exception as e:
    print("에러 발생:", e)
    # driver.quit()


가게 이름: 큐뮬러스
업종 카테고리: 샌드위치
별점: 방문자 리뷰 406
방문자 리뷰: 363
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 223
에러 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0056EBD3+24307]
	(No symbol) [0x004F8D74]
	(No symbol) [0x003DC323]
	(No symbol) [0x0041DC86]
	(No symbol) [0x0041DECB]
	(No symbol) [0x00413CF1]
	(No symbol) [0x0043FED4]
	(No symbol) [0x00413C14]
	(No symbol) [0x00440124]
	(No symbol) [0x00459579]
	(No symbol) [0x0043FC26]
	(No symbol) [0x0041219C]
	(No symbol) [0x0041311D]
	GetHandleVerifier [0x00818D93+2818227]
	GetHandleVerifier [0x0087542E+3196750]
	GetHandleVerifier [0x0086D9D2+3165426]
	GetHandleVerifier [0x0060DA70+675216]
	(No symbol) [0x00501B3D]
	(No symbol) [0x004FEA18]
	(No symbol) [0x004FEBB5]
	(No symbol) [0x004F1640]
	BaseThreadInitThunk [0x764CFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7742809E+286]
	RtlGetAppContainerNamedObjectPath [0x7742806E+238]

